### Fine-tune RO_BERTa transformer model
### Fine-tuning is tested on three custom paragraphs and relevant questions/answer pair.
#### We use library "deepset/roberta-base-squad2" from HuggingFace

In [ ]:
!pip install datasets
#!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq
!pip install fsspec
!pip install transformers
!pip install torch
!pip install accelerate==0.21.0
#!pip install accelerate
!pip install datasets
from datasets import Dataset, DatasetDict

from accelerate import Accelerator


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from transformers import *
from transformers import default_data_collator, Trainer
from transformers import AutoTokenizer, TrainingArguments, AutoModelForQuestionAnswering
import tensorflow as tf
#import collection
#from dataset import Dataset
import os
import pandas as pd

In [ ]:
#Fine the RO-BERT model on three different paragraphs and related questions and answer.
P1 = '''
             Anna Marie Frank was a German-born Jewish girl who
             kept a diary in which she documented life in hiding under Nazi persecution.
             She is a celebrated diarist who described everyday
             life from her family hiding place in an Amsterdam attic
             '''
P2 = '''
    Sir Isaac Newton FRS was an English mathematician, physicist, astronomer,
    alchemist, theologian, and author who was described in his
    time as a natural philosopher.
    He was a key figure in the Scientific Revolution
    and the Enlightenment that followed.
    '''
P3 = '''
    Srinivasa Ramanujan FRS was an Indian mathematician. Though he had almost
    no formal training in pure mathematics, he made substantial contributions
    to mathematical analysis, number theory, infinite series, and continued fractions,
    including solutions to mathematical problems then considered unsolvable.
     '''
df = pd.DataFrame()
df['id'] = ['903deec17', 'd9841668c', '29d154b56']
df['context'] = [P1, P2, P3]

df['question'] = ['Why is Anne Frank famous? ',  'What are contributions from Newton', 'What topics did Ramanujan worked on?']

df['answer_text'] = ['for documenting her life during Nazi period', 'contributed in the field of mathematics, physics and astronomy',
                     'mathematical analysis, number theory, infinite series, and continued fractions']
df['answer_start'] = [ 94, 36 , 158]
df['language'] = [ 'english', 'english', 'english']

df

,id,context,question,answer_text,answer_start,language
0,903deec17,\n Anna Marie Frank was a German-b...,Why is Anne Frank famous?,for documenting her life during Nazi period,94,english
1,d9841668c,\n Sir Isaac Newton FRS was an English math...,What are contributions from Newton,"contributed in the field of mathematics, physi...",36,english
2,29d154b56,\n Srinivasa Ramanujan FRS was an Indian ma...,What topics did Ramanujan worked on?,"mathematical analysis, number theory, infinite...",158,english


In [ ]:
#Create a new columns with merging ['answer_start', 'answer_text']
def convert_answers(r):
    start = r[0]
    text = r[1]
    return {'answer_start': [start], 'text': [text] }


df_new = df[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

df['answers'] = df_new
df

,id,context,question,answer_text,answer_start,language,answers
0,903deec17,\n Anna Marie Frank was a German-b...,Why is Anne Frank famous?,for documenting her life during Nazi period,94,english,"{'answer_start': [94], 'text': ['for documenti..."
1,d9841668c,\n Sir Isaac Newton FRS was an English math...,What are contributions from Newton,"contributed in the field of mathematics, physi...",36,english,"{'answer_start': [36], 'text': ['contributed i..."
2,29d154b56,\n Srinivasa Ramanujan FRS was an Indian ma...,What topics did Ramanujan worked on?,"mathematical analysis, number theory, infinite...",158,english,"{'answer_start': [158], 'text': ['mathematical..."


In [ ]:
train_dataset = Dataset.from_pandas(df)
valid_dataset = Dataset.from_pandas(df)
train_dataset

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language', 'answers'],
    num_rows: 3
})

In [ ]:
train_dataset[0]


{'id': '903deec17',
 'context': '\n             Anna Marie Frank was a German-born Jewish girl who \n             kept a diary in which she documented life in hiding under Nazi persecution. \n             She is a celebrated diarist who described everyday \n             life from her family hiding place in an Amsterdam attic \n             ',
 'question': 'Why is Anne Frank famous? ',
 'answer_text': 'for documenting her life during Nazi period',
 'answer_start': 94,
 'language': 'english',
 'answers': {'answer_start': [94],
  'text': ['for documenting her life during Nazi period']}}

In [ ]:
tokenizer  = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
batch_size = 4
max_length = 384
doc_stride = 128
num_train_epochs = 10
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
pad_on_right

True

In [ ]:
def prepare_train_features( df_input ):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    # examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        df_input["question" if pad_on_right else "context"],
        df_input["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    print(tokenized_examples)

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    print(sample_mapping)
    for a in sample_mapping:
      print(a)
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")
    print(offset_mapping)
    for a in offset_mapping:
      print(a)
    # The offset

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = df_input["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
    #'''
    return tokenized_examples

In [ ]:
#Tokenize data frame
#The map function: It allows you to apply a processing function to
# each example in a dataset, independently or in batches
# thereby speeding up the operation

tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True)#, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True)#, remove_columns=train_dataset.column_names)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

{'input_ids': [[0, 7608, 16, 7896, 3848, 3395, 116, 1437, 2, 2, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 7687, 9340, 3848, 21, 10, 1859, 12, 5400, 4586, 1816, 54, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1682, 10, 25694, 11, 61, 79, 12404, 301, 11, 9646, 223, 12281, 19913, 4, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 264, 16, 10, 5417, 2269, 33547, 54, 1602, 7476, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 301, 31, 69, 284, 9646, 317, 11, 41, 16342, 33093, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

{'input_ids': [[0, 7608, 16, 7896, 3848, 3395, 116, 1437, 2, 2, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 7687, 9340, 3848, 21, 10, 1859, 12, 5400, 4586, 1816, 54, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1682, 10, 25694, 11, 61, 79, 12404, 301, 11, 9646, 223, 12281, 19913, 4, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 264, 16, 10, 5417, 2269, 33547, 54, 1602, 7476, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 301, 31, 69, 284, 9646, 317, 11, 41, 16342, 33093, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
#len(tokenized_train_ds['input_ids'])

df = pd.DataFrame()

Label = []
for a in tokenized_train_ds:
    print(a)
    df = df.append(a, ignore_index=True)

df
#'''


{'id': '903deec17', 'context': '\n             Anna Marie Frank was a German-born Jewish girl who \n             kept a diary in which she documented life in hiding under Nazi persecution. \n             She is a celebrated diarist who described everyday \n             life from her family hiding place in an Amsterdam attic \n             ', 'question': 'Why is Anne Frank famous? ', 'answer_text': 'for documenting her life during Nazi period', 'answer_start': 94, 'language': 'english', 'answers': {'answer_start': [94], 'text': ['for documenting her life during Nazi period']}, 'input_ids': [0, 7608, 16, 7896, 3848, 3395, 116, 1437, 2, 2, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 7687, 9340, 3848, 21, 10, 1859, 12, 5400, 4586, 1816, 54, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1682, 10, 25694, 11, 61, 79, 12404, 301, 11, 9646, 223, 12281, 19913, 4, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437,

<ipython-input-32-5f77a1bd9abc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a, ignore_index=True)
<ipython-input-32-5f77a1bd9abc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a, ignore_index=True)
<ipython-input-32-5f77a1bd9abc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a, ignore_index=True)


,id,context,question,answer_text,answer_start,language,answers,input_ids,attention_mask,start_positions,end_positions
0,903deec17,\n Anna Marie Frank was a German-b...,Why is Anne Frank famous?,for documenting her life during Nazi period,94,english,"{'answer_start': [94], 'text': ['for documenti...","[0, 7608, 16, 7896, 3848, 3395, 116, 1437, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",51,59
1,d9841668c,\n Sir Isaac Newton FRS was an English math...,What are contributions from Newton,"contributed in the field of mathematics, physi...",36,english,"{'answer_start': [36], 'text': ['contributed i...","[0, 2264, 32, 5694, 31, 10793, 2, 2, 50118, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",19,34
2,29d154b56,\n Srinivasa Ramanujan FRS was an Indian ma...,What topics did Ramanujan worked on?,"mathematical analysis, number theory, infinite...",158,english,"{'answer_start': [158], 'text': ['mathematical...","[0, 2264, 7614, 222, 3513, 11115, 9193, 1006, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",50,65


In [ ]:
for a in tokenized_train_ds:
  Label = a
  print(a)
  for b in a:
    print(b)

{'id': '903deec17', 'context': '\n             Anna Marie Frank was a German-born Jewish girl who \n             kept a diary in which she documented life in hiding under Nazi persecution. \n             She is a celebrated diarist who described everyday \n             life from her family hiding place in an Amsterdam attic \n             ', 'question': 'Why is Anne Frank famous? ', 'answer_text': 'for documenting her life during Nazi period', 'answer_start': 94, 'language': 'english', 'answers': {'answer_start': [94], 'text': ['for documenting her life during Nazi period']}, 'input_ids': [0, 7608, 16, 7896, 3848, 3395, 116, 1437, 2, 2, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 7687, 9340, 3848, 21, 10, 1859, 12, 5400, 4586, 1816, 54, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1682, 10, 25694, 11, 61, 79, 12404, 301, 11, 9646, 223, 12281, 19913, 4, 1437, 50118, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 1437,

In [ ]:
#Configure the training parameters
# The data_collator automatically performs padding on the model inputs in a batch
# to the length of the most extended example in the dataset that elimincates the
# need to set a maximum sequence length that is usually fixed

data_collator = default_data_collator

model = AutoModelForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

args = TrainingArguments(
    f"OutputROBERTDIR",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
!accelerate --version

2023-07-13 15:39:34.426743: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: accelerate <command> [<args>]
Accelerate CLI tool: error: unrecognized arguments: --version


In [ ]:
#Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,6.421059
2,No log,6.421059
3,No log,6.421059
4,No log,6.421059
5,No log,6.421059
6,No log,6.421059
7,No log,6.421059
8,No log,5.408647
9,No log,5.408647
10,No log,5.408647


TrainOutput(global_step=10, training_loss=0.7551511764526367, metrics={'train_runtime': 257.3296, 'train_samples_per_second': 0.117, 'train_steps_per_second': 0.039, 'total_flos': 5879177026560.0, 'train_loss': 0.7551511764526367, 'epoch': 10.0})

In [ ]:
#Save the model
trainer.save_model("final_model_ROBert_trained")


In [ ]:
!du -sh final_model_ROBert_trained

477M	final_model_ROBert_trained


In [ ]:

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

###
validation_features = valid_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=valid_dataset.column_names
)
len(validation_features)

valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

raw_predictions = trainer.predict(valid_feats_small)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
raw_predictions

PredictionOutput(predictions=(array([[ 0.73876405, -7.3731575 , -9.026638  , ..., -9.644873  ,
        -9.644873  , -9.644873  ],
       [ 0.68632084, -5.87234   , -7.730211  , ..., -9.479424  ,
        -9.479424  , -9.479424  ],
       [ 1.6337625 , -8.532843  , -8.662145  , ..., -9.512573  ,
        -9.512573  , -9.512573  ]], dtype=float32), array([[ 1.276333 , -7.139175 , -7.6984153, ..., -8.223457 , -8.223457 ,
        -8.223457 ],
       [ 1.0165164, -7.643339 , -8.026371 , ..., -8.570888 , -8.570888 ,
        -8.570888 ],
       [ 1.9144971, -8.078679 , -7.400689 , ..., -8.717888 , -8.717888 ,
        -8.717888 ]], dtype=float32)), label_ids=None, metrics={'test_runtime': 4.8421, 'test_samples_per_second': 0.62, 'test_steps_per_second': 0.207})

In [ ]:
import collections

examples = valid_dataset
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

features_per_example

defaultdict(list, {0: [0], 1: [1], 2: [2]})

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(valid_dataset, validation_features, raw_predictions.predictions)
prediction = pd.DataFrame([{"questions":x1['question'], "pred_answer":x2} for x1, x2 in zip(valid_dataset, [i for i in final_predictions.values()])])


Post-processing 3 example predictions split into 3 features.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
prediction

,questions,pred_answer
0,Why is Anne Frank famous?,diarist
1,What are contributions from Newton,"mathematician, physicist, astronomer, \n al..."
2,What topics did Ramanujan worked on?,"mathematical analysis, number theory, infinite..."


import numpy as np
from google.colab import autoviz
df_5089267707001084717 = autoviz.get_registered_df('df_5089267707001084717')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5089267707001084717, *['questions'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5089267707001084717 = autoviz.get_registered_df('df_5089267707001084717')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_5089267707001084717, *['pred_answer'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5089267707001084717 = autoviz.get_registered_df('df_5089267707001084717')

def heatmap(df, x_colname, y_colname, figsize=(2, 2), mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=figsize)
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(df_5089267707001084717, *['questions', 'pred_answer'], **{})
chart